<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Load data to Vantage</b>
</header>

<p style = 'font-size:16px;font-family:Arial'>Welcome to this introductory guide. This guide will walk you through steps  are needed to load data in Teradata from ipython SQL kernel.  A more extensive notebook with more commands can be found online in the Teradata Vantage Modules for Jupyter <a href = 'https://teradata.github.io/jupyterextensions/#/'>Here.</a></p>

<hr>
<p style = 'font-size:16px;font-family:Arial'>For help on available "magic" commands in the Teradata SQL plugin, run the following cell:</p>

<h1 style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Connect to Vantage</h1>
<p style = 'font-size:18px;font-family:Arial;'> <b>Let's start by connecting to the Teradata system </b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [ ]:
%connect local, hidewarnings=true

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Load data from file present locally </b></p>

<p style = 'font-size:16px;font-family:Arial'>Apart from the standard Teradata utilities for data load i.e TPT; Teradata SQL plugin for jupyter has a magic command to load data into the database. 

In [ ]:
%help dataload

In [ ]:
CREATE MULTISET TABLE DEMO_USER.Retail_Data 
       (InvoiceNo VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        StockCode VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Description VARCHAR(40) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Quantity BIGINT,
        InvoiceDate TIMESTAMP(6),
        UnitPrice FLOAT,
        CustomerID FLOAT,
        Country VARCHAR(25) CHARACTER SET UNICODE NOT CASESPECIFIC)
  PRIMARY INDEX ( InvoiceNo );

In [ ]:
%dataload DATABASE=demo_user, TABLE=Retail_Data, skiprows=1,FILEPATH=Getting_Started/Data_Load/Retail_Data_sample.csv

<p style = 'font-size:16px;font-family:Arial'>Let's check the count and sample records loaded. 

In [ ]:
select count(*) from DEMO_USER.Retail_Data;

In [ ]:
select * from DEMO_USER.Retail_Data sample 10;

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Load data from file present in cloud </b></p>

<p style = 'font-size:16px;font-family:Arial'>We can access and load files present in cloud via Vantage's NOS (Native Object Store). For this we need the access id and key for authentication and the location off the file. Please refer <a href ='https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-VantageTM-Native-Object-Store-Getting-Started-Guide-17.20/Reading-Data/Examples-For-Business-Analysts/Sampling-External-Data-In-Place-Using-READ_NOS'>NOS user guide</a>  for more information.<br>
  First of all we need to create Authorization to access the file.  

In [ ]:
/* The command to create authorization is below, we already have the authorization created. */

/*
CREATE AUTHORIZATION <name> AS DEFINER TRUSTED
USER '<access_key>'
PASSWORD '<password>';

*/

In [ ]:
SHOW AUTHORIZATION gs_tables_db.auth

<p style = 'font-size:16px;font-family:Arial'>
Our file Retail_Data_sample.csv is located at /gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/<br>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Sampling External Data</b><br>
<p style = 'font-size:16px;font-family:Arial'>We can preview the contents of an external object store by using below command.<br><b>Using Read_NOS    

In [ ]:
SELECT TOP 2 * FROM (
LOCATION='/gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/'
AUTHORIZATION=gs_tables_db.auth 
) AS D;

<p style = 'font-size:16px;font-family:Arial'> Please keep in mind:
<ul style = 'font-size:16px;font-family:Arial'><li>TOP does not return the same results each time it is run.</li>
<li>TOP may return records with only a subset of attributes because not all records in different objects contain all of the same attributes.</li>
    </ul>

<p style = 'font-size:16px;font-family:Arial'><b>Using Foreign Table</b>

In [ ]:
CREATE FOREIGN TABLE Retail_Data_FT
, EXTERNAL SECURITY  gs_tables_db.auth
USING (LOCATION('/gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/') );

In [ ]:
select top 5* from Retail_Data_FT;

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Loading External Data in permanent table</b><br>
<p style = 'font-size:16px;font-family:Arial'>If we want to copy data from cloud to permanent table in database we can use the below command

In [ ]:
CREATE MULTISET TABLE DEMO_USER.Retail_Data_Perm 
       (InvoiceNo VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        StockCode VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Description VARCHAR(40) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Quantity BIGINT,
        InvoiceDate TIMESTAMP(6),
        UnitPrice FLOAT,
        CustomerID FLOAT,
        Country VARCHAR(25) CHARACTER SET UNICODE NOT CASESPECIFIC)
  PRIMARY INDEX ( InvoiceNo );

In [ ]:
Insert DEMO_USER.Retail_Data_Perm 
SELECT InvoiceNo 
,StockCode 
,Description 
,Quantity 
,InvoiceDate 
,UnitPrice 
,CustomerID 
,Country
FROM (
LOCATION='/gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/'
AUTHORIZATION= gs_tables_db.auth
) AS d


In [ ]:
select top 5* from Retail_Data_Perm;

 <p style = 'font-size:16px;font-family:Arial'> To load other file formats like json and parquet please refer <a href ='https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-VantageTM-Native-Object-Store-Getting-Started-Guide-17.20/Reading-Data/Examples-For-Business-Analysts/Sampling-External-Data-In-Place-Using-READ_NOS'>NOS user guide</a> 

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Clean up</b>

In [ ]:
DROP TABLE DEMO_USER.Retail_Data;

In [ ]:
DROP TABLE DEMO_USER.Retail_Data_FT;

In [ ]:
DROP TABLE DEMO_USER.Retail_Data_Perm;

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>